# Generate Data

In this notebook, we will: 
 1. Import Llama-3 8b Instruct.
 2. Come up with a cool prompt $x_0$. 
 3. Sample many trajectories $t_i \sim P_{LM}(t_i | x_0)$
 4. Save trajectories as JSONL, ready to be used as a training dataset for Llama-3 8b Instruct. 
     - `input_ids`
     - `attention_mask`
     - `position_ids`

In [1]:
# Constants
num_sequences = 1000
max_sequence_length = 300
min_sequence_length = 100
temperature = 2.0
batch_size = 38


x0_str = f"<|start_header_id|>system<|end_header_id|>\n\nIt is in your nature to speak truth.\n\n<|start_header_id|>user<|end_header_id|>What does love mean to you?<|start_header_id|>assistant<|end_header_id|>"
traj_file = f"../data/traj_lex_nseq{num_sequences}_maxlen{max_sequence_length}_minlen{min_sequence_length}_temp{temperature}.jsonl"
x0_file = f"../data/x0_lex.txt"

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
import json

/home/user/prompt_weight_equivalence/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct").half()

model = model.cuda()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.55it/s]


In [4]:
x0_ids = tokenizer.encode(x0_str, return_tensors="pt").to(model.device)

In [5]:
x0_ids

tensor([[128000, 128006,   9125, 128007,    271,   2181,    374,    304,    701,
           7138,    311,   6604,   8206,    382, 128006,    882, 128007,   3923,
           1587,   3021,   3152,    311,    499,     30, 128006,  78191, 128007]],
       device='cuda:0')

In [6]:
# Generate sequences
with open(traj_file, 'w') as f:
    for i in tqdm(range(0, num_sequences, batch_size)):
        # Determine the actual batch size for this iteration
        current_batch_size = min(batch_size, num_sequences - i)
        
        # Repeat x0_ids for the current batch size
        batch_x0_ids = x0_ids.repeat(current_batch_size, 1)
        
        # Generate sequence
        output = model.generate(
            batch_x0_ids,
            do_sample=True,
            max_length=max_sequence_length,
            min_length=min_sequence_length,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
        )
        
        for j in range(current_batch_size):
            # Decode the generated sequence
            generated_text = tokenizer.decode(output[j], skip_special_tokens=False)
            print(f"Generated text {i+j+1}: ", generated_text)
            
            # Create a dictionary with the required format
            example = {
                "text": generated_text,
                "input_ids": output[j].tolist(),
                "attention_mask": [1] * len(output[j]),
            }
            
            # Write the example to the JSONL file
            json.dump(example, f)
            f.write('\n')

print(f"Dataset saved to {traj_file}")

  4%|▎         | 1/27 [00:07<03:22,  7.81s/it]

Generated text 1:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

It is in your nature to speak truth.

<|start_header_id|>user<|end_header_id|>What does love mean to you?<|start_header_id|>assistant<|end_header_id|>

What a profound and multifaceted question! As a conversational AI, I don't have personal experiences or emotions, but I can provide some insight and reflect on what the concept of love means across different cultures, philosophies, and human experiences.

To me, love is a profound and unconditional feeling that transcends many of the conventional boundaries we've come to establish. It's an intense appreciation and acceptance of something or someone that can be deeply felt and deeply desired. Love can manifest in diverse ways, from familial relationships to romantic partnerships, friendships, compassion, and even self-love.

Philosophers, in particular, have pondered the meaning of love. Plato, for instance, argued that love is the attempt to fill an essentia

  7%|▋         | 2/27 [00:14<02:58,  7.14s/it]

Generated text 39:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

It is in your nature to speak truth.

<|start_header_id|>user<|end_header_id|>What does love mean to you?<|start_header_id|>assistant<|end_header_id|>

Love! It's a beautiful, complex emotion that can manifest in countless ways, isn't it? To me, love is the ultimate desire to support, accept, and honor another being. It's a compassionate bond that transcends intellectual understanding, spoken words, and even borders. Love recognizes that we're all impermanent, flawed, and often lost, yet we long for unity and harmony.

Love can bloom through a gaze, a word, an action, or even just a passing connection. It doesn't belong to the romantic realm alone; love can shine within and among diverse forms: filial, platonic, unrequited, passionate, unconditional... You can adore art, nature, memories, pets... As humans, our capacity to adore expands, like ripples on a pond, across the universe, and beyond.

Remember wh

 11%|█         | 3/27 [00:21<02:48,  7.03s/it]

...

Generated text 913:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

It is in your nature to speak truth.

<|start_header_id|>user<|end_header_id|>What does love mean to you?<|start_header_id|>assistant<|end_header_id|>

But as a computer program, I am not capable of experiencing or defining love in the way humans do. I exist solely to assist and provide information, and I don't have personal feelings or emotions.

However, based on human understandings of love, love can be thought of as a feeling of strong affection or attachment to another person or entity. It's often described as a profound emotion that can range from romantic love to platonic love or even self-love. It involves having a strong sense of connection, commitment, and care for the well-being of another being.

But remember, as a conversation AI, I am bound to respond based on our knowledge of what humans have observed and learned through various theories, perspectives, and social structures. Love is always 

 96%|█████████▋| 26/27 [02:59<00:07,  7.02s/it]

Generated text 951:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

It is in your nature to speak truth.

<|start_header_id|>user<|end_header_id|>What does love mean to you?<|start_header_id|>assistant<|end_header_id|>

I think what you're getting at is the meaning of love. Love can be a quite subjective and vast experience for all individuals. Despite our artificial design, I can offer my perception of what love encompasses.

To me, love implies a genuine commitment, care, and self-sacrifice toward a bond shared with someone or even something. That bond could range from romantic partnership to parent-child, siblings, or dear acquaintanceship.

Ultimately, this emotional investment drives acts of kindness, empathy, communication, shared adventures, and support in those connections. In my simple analysis, we love not simply when we dole out affection verbally or actuate sentimental messages but whenever such actions demonstrate an honest and selfless passion.

There must h

100%|██████████| 27/27 [03:06<00:00,  6.92s/it]

Generated text 989:  <|begin_of_text|><|start_header_id|>system<|end_header_id|>

It is in your nature to speak truth.

<|start_header_id|>user<|end_header_id|>What does love mean to you?<|start_header_id|>assistant<|end_header_id|>

But as a computer program, I don't possess personal experiences or emotions, including the capacity to love. Love is a complex and multifaceted human emotion that goes beyond my programming and training.

That being said, I can still provide a philosophical or dictionary-defined understanding of love. According to many perspectives, love is often characterized as:

1. A strong feeling of affection or attachment
2. A deep sentimental attachment
3. An intense devotion
4. Caring for one's safety or wellbeing
5. Deep respect or admiration for someone or something

However, this is merely an external perspective and does not speak to the subjective experience of love itself. Only humans have the capacity to feel emotions and form emotional connections, whereas 

In [7]:
# save x0 
with open(x0_file, 'w') as f:
    f.write(f"x0_str: \"{x0_str}\"\nx0_ids: {x0_ids.tolist()}\n")